In [42]:
import os
import json
from dotenv import load_dotenv
from IPython.display import display,Markdown,update_display
from openai import OpenAI
from scraper import fetch_website_contents,fetch_website_links

In [17]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_key')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print('API Key looks good so far')
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API Key looks good so far


In [18]:
MODEL="gpt-5-nano"
open_ai = OpenAI()

In [19]:
links=fetch_website_links("https://www.smxtech.com/")
links

['/about-us/news/smx-announces-acquisition-of-cbeyondata/',
 'https://www.smxtech.com/',
 '#',
 '#',
 'https://www.smxtech.com/solutions/cybersecurity/',
 'https://www.smxtech.com/solutions/data-analytics/',
 'https://www.smxtech.com/solutions/digital-transformation-cloud-automation/',
 'https://www.smxtech.com/solutions/enterprise-applications/',
 'https://www.smxtech.com/solutions/mission-operations-and-intelligence/',
 'https://www.smxtech.com/solutions/modeling-simulation-training/',
 '#',
 'https://www.smxtech.com/solutions/smx-elevate-overview/',
 'https://www.smxtech.com/solutions/smx-elevate-fast-track/',
 'https://www.smxtech.com/solutions/smx-elevate-fast-track/isvs/',
 'https://www.smxtech.com/solutions/smx-elevate-fast-track/government/',
 'https://www.smxtech.com/solutions/smx-elevate-fm/',
 'https://www.smxtech.com/solutions/smx-elevate-isr/',
 'https://www.smxtech.com/solutions/smx-elevate-netedge/',
 'https://www.smxtech.com/solutions/smx-elevate-sensing/',
 'https://ww

In [20]:
link_system_prompt=""" 
you are provided with a list of links found on a webpage.
you are able to decide which of the links would be most relevant to include in a broucher about the company,
such as links to an About page, or a company page, or careers/Jobs pages.
you should respond in JSON as in this example:

{
    "links":[
        {"type":"about page","url":"https://full.url/goes/here/about"},
        {"type":"careers page", "url":"https://another.full.url/careers"}
    ]
}
"""

In [21]:
def get_links_user_prompt(url):
    user_prompt=f""" 
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (Some might be relative links):    
    
"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_links_user_prompt("https://www.smxtech.com/"))

 
Here is the list of links on the website https://www.smxtech.com/ -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (Some might be relative links):    

/about-us/news/smx-announces-acquisition-of-cbeyondata/
https://www.smxtech.com/
#
#
https://www.smxtech.com/solutions/cybersecurity/
https://www.smxtech.com/solutions/data-analytics/
https://www.smxtech.com/solutions/digital-transformation-cloud-automation/
https://www.smxtech.com/solutions/enterprise-applications/
https://www.smxtech.com/solutions/mission-operations-and-intelligence/
https://www.smxtech.com/solutions/modeling-simulation-training/
#
https://www.smxtech.com/solutions/smx-elevate-overview/
https://www.smxtech.com/solutions/smx-elevate-fast-track/
https://www.smxtech.com/solutions/smx-elevate-fast-track/isvs/
https://www.smxtech.com/solutions/smx-elevate-fast-track/governme

In [30]:
def select_relevant_links(url):
    response=open_ai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system",'content':link_system_prompt},
            {"role":"user",'content':get_links_user_prompt(url)}
        ],
        response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    links=json.loads(result)
    return result

In [25]:
def select_relevant_links(url):
    response = open_ai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [28]:
select_relevant_links("https://edwarddonner.com/")

'{\n  "links": [\n    {"type": "company page", "url": "https://edwarddonner.com/"},\n    {"type": "about page", "url": "https://edwarddonner.com/about-me-and-about-nebula/"},\n    {"type": "LinkedIn page", "url": "https://www.linkedin.com/in/eddonner/"},\n    {"type": "Twitter page", "url": "https://twitter.com/edwarddonner"},\n    {"type": "Facebook page", "url": "https://www.facebook.com/edward.donner.52"}\n  ]\n}'

In [29]:
select_relevant_links("https://www.smxtech.com/")

'{\n  "links": [\n    {"type": "home page", "url": "https://www.smxtech.com/"},\n    {"type": "about page", "url": "https://www.smxtech.com/about-us/"},\n    {"type": "awards page", "url": "https://www.smxtech.com/about-us/awards-recognition/"},\n    {"type": "clients page", "url": "https://www.smxtech.com/about-us/clients/"},\n    {"type": "partners page", "url": "https://www.smxtech.com/about-us/partners/"},\n    {"type": "leaders page", "url": "https://www.smxtech.com/about-us/leaders/"},\n    {"type": "quality certifications", "url": "https://www.smxtech.com/about-us/quality-business-certifications/"},\n    {"type": "contracting vehicles", "url": "https://www.smxtech.com/contracting-vehicles/"},\n    {"type": "doing business page", "url": "https://www.smxtech.com/doing-business/"},\n    {"type": "contact page", "url": "https://www.smxtech.com/contact-us/"},\n    {"type": "resources page", "url": "https://www.smxtech.com/resources/"},\n    {"type": "customer spotlights", "url": "htt

In [31]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = open_ai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [32]:
select_relevant_links("https://www.smxtech.com/")

Selecting relevant links for https://www.smxtech.com/ by calling gpt-5-nano
Found 36 relevant links


{'links': [{'type': 'home page', 'url': 'https://www.smxtech.com/'},
  {'type': 'about page', 'url': 'https://www.smxtech.com/about-us/'},
  {'type': 'news page',
   'url': 'https://www.smxtech.com/about-us/news/smx-announces-acquisition-of-cbeyondata/'},
  {'type': 'news page',
   'url': 'https://www.smxtech.com/about-us/news/energage-names-smx-a-winner-of-the-2025-top-workplaces-usa/'},
  {'type': 'news page',
   'url': 'https://www.smxtech.com/about-us/news/smx-elevate-financial-management%e2%84%a0-now-available-on-the-treasurys-fm-qsmo-marketplace/'},
  {'type': 'awards page',
   'url': 'https://www.smxtech.com/about-us/awards-recognition/'},
  {'type': 'clients page', 'url': 'https://www.smxtech.com/about-us/clients/'},
  {'type': 'partners page',
   'url': 'https://www.smxtech.com/about-us/partners/'},
  {'type': 'leadership page',
   'url': 'https://www.smxtech.com/about-us/leaders/'},
  {'type': 'certifications page',
   'url': 'https://www.smxtech.com/about-us/quality-business

In [34]:
def fetch_page_and_all_relevant_links(url):
    contents=fetch_website_contents(url)
    relevant_links=select_relevant_links(url)
    result= f"## Landing page:\n\n{contents} \n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [35]:
fetch_page_and_all_relevant_links("https://www.smxtech.com/")

Selecting relevant links for https://www.smxtech.com/ by calling gpt-5-nano
Found 13 relevant links


'## Landing page:\n\nSMX - Harness Technology to Realize Your Digital Future\n\nSMX announces the acquisition of cBEYONData, enhancing our digital transformation capabilities and amplifying our ability to improve the business of government.\nSolutions\nAll Solutions\nCybersecurity\nData & Analytics\nDigital Transformation\nEnterprise Applications\nMission Operations & Intelligence\nModeling, Simulation & Training\nAccelerators\nSMX Elevate\nElevate Fast Track\nISVs\nGovernment\nElevate FM\nElevate ISR\nElevate NetEdge\nElevate Sensing\nGenerative AI\nInsights\nNews\nCustomer Spotlights\nResources\nEvents\nCareers\nOpen Positions\nAll Career Opportunities\nDMV Cleared Roles\nHawaii Careers\nSouthern Maryland Careers\nBenefits\nSkillBridge Fellowship Program\nAbout\nAbout Us\nAwards & Recognition\nClients\nContracting Vehicles\nPartners\nLeadership & Governance\nQuality & Business Certifications\nDoing Business\nContact\nSelect Page\nOur QSMO-approved Core FS solution enhances federal fi

In [36]:
print(fetch_page_and_all_relevant_links("https://www.smxtech.com/"))

Selecting relevant links for https://www.smxtech.com/ by calling gpt-5-nano
Found 25 relevant links
## Landing page:

SMX - Harness Technology to Realize Your Digital Future

SMX announces the acquisition of cBEYONData, enhancing our digital transformation capabilities and amplifying our ability to improve the business of government.
Solutions
All Solutions
Cybersecurity
Data & Analytics
Digital Transformation
Enterprise Applications
Mission Operations & Intelligence
Modeling, Simulation & Training
Accelerators
SMX Elevate
Elevate Fast Track
ISVs
Government
Elevate FM
Elevate ISR
Elevate NetEdge
Elevate Sensing
Generative AI
Insights
News
Customer Spotlights
Resources
Events
Careers
Open Positions
All Career Opportunities
DMV Cleared Roles
Hawaii Careers
Southern Maryland Careers
Benefits
SkillBridge Fellowship Program
About
About Us
Awards & Recognition
Clients
Contracting Vehicles
Partners
Leadership & Governance
Quality & Business Certifications
Doing Business
Contact
Select Page
Ou

In [37]:
brochure_system_prompt="""
you are an assistant that analyzes the contents of serveral relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [38]:
def get_brochure_user_prompt(company_name,url):
    user_prompt=f""" 
You are looking at a company called {company_name} 
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_0000]
    return user_prompt


In [39]:
get_brochure_user_prompt("SMX","https://www.smxtech.com/")

Selecting relevant links for https://www.smxtech.com/ by calling gpt-5-nano
Found 36 relevant links


' \nYou are looking at a company called SMX \nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing page:\n\nSMX - Harness Technology to Realize Your Digital Future\n\nSMX announces the acquisition of cBEYONData, enhancing our digital transformation capabilities and amplifying our ability to improve the business of government.\nSolutions\nAll Solutions\nCybersecurity\nData & Analytics\nDigital Transformation\nEnterprise Applications\nMission Operations & Intelligence\nModeling, Simulation & Training\nAccelerators\nSMX Elevate\nElevate Fast Track\nISVs\nGovernment\nElevate FM\nElevate ISR\nElevate NetEdge\nElevate Sensing\nGenerative AI\nInsights\nNews\nCustomer Spotlights\nResources\nEvents\nCareers\nOpen Positions\nAll Career Opportunities\nDMV Cleared Roles\nHawaii Careers\nSouthern Maryland Careers\nBenefits\nSkillBridge Fellowship Program\nAbout\nAbout Us\nA

In [40]:
def create_brochure(company_name,url):
    response=open_ai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role":"system","content":brochure_system_prompt},
            {"role":"user","content":get_brochure_user_prompt(company_name,url)}
        ]
    )
    result=response.choices[0].message.content
    display(Markdown(result))

In [41]:
create_brochure("SMX","https://www.smxtech.com/")

Selecting relevant links for https://www.smxtech.com/ by calling gpt-5-nano
Found 23 relevant links


# SMX Company Brochure

---

## About SMX

**SMX** is a leading technology company dedicated to harnessing the transformative power of technology to help public and private sector enterprises achieve mission success. With over 30 years of combined experience through its legacy companies, SMX brings together a collective of technical and domain experts specializing in delivering innovative, secure, and mission-critical solutions. The company plays a vital role in national security and government modernization efforts, as well as supporting commercial enterprises.

SMX operates with a strong mission-driven culture focused on innovation, collaboration, and integrity. Their unique delivery model combines deep technical expertise with forward-looking insights, empowering customers to overcome complex challenges and realize digital transformation at scale.

---

## Our Solutions

SMX offers a comprehensive portfolio of advanced technology solutions designed to create tangible business and mission impact:

- **Cybersecurity**: State-of-the-art protection for data, applications, and people, including managed security services, continuous monitoring, vulnerability remediation, and compliance automation.
  
- **Data & Analytics**: Enables secure, data-centric analytics with AI, ML, RPA, and IoT technologies to drive faster, smarter decision-making and operational agility.
  
- **Digital Transformation**: Outcome-focused automation and cloud architecture optimization to accelerate secure mission delivery.
  
- **Enterprise Applications**: Customized application services tailored to mission requirements.
  
- **Mission Operations & Intelligence**: Expertise and leadership supporting national security initiatives in defense, intelligence, and homeland security sectors with C5ISR capabilities, data science, and operational support worldwide.
  
- **Modeling, Simulation & Training**: Realistic simulation environments, digital twins, and training technology to enhance predictive analysis and operational readiness.
  
- **Accelerators (SMX Elevate)**: A technology-enabled approach to removing barriers in missions through intelligent automation, generative AI, and rapid innovation.

---

## Key Highlights

- **Acquisition of cBEYONData**: Strengthened SMX's digital transformation capabilities and expanded offerings in budget management and financial analytics for the Department of Defense and federal agencies.
  
- **SMX Elevate AI Labs℠**: Offers a secure enclave for safe AI experimentation, protecting proprietary information and enabling accountable AI model development.
  
- **Customer Impact**: Trusted by Department of Defense branches (Army, Navy, Air Force, Marine Corps), federal civilian agencies (Dept of Treasury, Dept of Interior), and leading commercial clients such as Mayo Clinic, Pepperidge Farm, and Allergan.

---

## Company Culture & Recognition

SMX prides itself on a deeply collaborative and inclusive culture that encourages curiosity, ownership, and pushing boundaries to solve complex problems. Respect for diverse points of view and teamwork are foundational, empowering employees to innovate and make a meaningful impact.

**Awards & Recognition:**

- Winner of the Energage Top Workplaces USA Award for three consecutive years (2023-2025).
- Recognized as a “#1 Best Firm to Work For,” “Top Technology Firm,” and “Employer of Choice.”
- Awarded the Employer Support of the Guard and Reserve (ESGR) Patriot Award for service to military reserve members.
- Multiple cultural excellence awards covering compensation, employee appreciation, well-being, innovation, pro development, and work-life balance.

---

## Careers at SMX

At SMX, you’re not just taking a job—you’re joining a community that values your skills, ideas, and career growth. From entry-level roles to senior leadership, SMX offers opportunities to work on meaningful projects, including national security missions and cutting-edge digital transformation.

Current career initiatives include:

- **SkillBridge Fellowship Program**: Helping transitioning military service members gain civilian workforce experience while still on active duty.
- Diverse roles across technical, operational, and business areas with locations nationwide including Maryland, Virginia, Hawaii, and other states.
- Competitive benefits including comprehensive health plans, 401(k) with matching, wellness programs, paid time off, and work-life integration support.

---

## Partnerships & Certifications

SMX maintains strong partnerships with leading technology providers:

- AWS Premier Consulting Partner
- Microsoft Azure Expert Managed Service Provider
- Google Cloud Partner
- Oracle Partner
- Relationships with companies such as Dell, IBM, Fortinet, Palo Alto Networks, Salesforce, ServiceNow, Snowflake, and more.

**Quality & Compliance Certifications:**

- ISO 9001 (since 2005)
- AS9100 (since 2019)
- ISO 20000 (since 2011)
- ISO 27001 (since 2016)
- CMMI SVC & DEV Maturity Level 3

---

## Contact & Locations

**Corporate Headquarters:**  
Hollywood, Maryland  
Certified for ISO 9001, AS9100D, and CMMI Maturity Level 3

**Other Key Location:**  
Herndon, Virginia  
Hold ISO 20000 and 27001 certifications

**General Inquiries:** [info@smx.com](mailto:info@smx.com)  
**Careers:** [careers@smx.com](mailto:careers@smx.com)  
**Sales & Solutions:** [sales@smx.com](mailto:sales@smx.com)

---

## Shared Vision. Outcome Assured.®

SMX is dedicated to accelerating mission and business impact securely and at scale, transforming tomorrow’s challenges into assured success today.

---

For more information, visit [www.smx.com](http://www.smx.com) and join the journey to realize your digital future.

In [43]:
def stream_brochure(company_name, url):
    stream = open_ai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [45]:
stream_brochure("SMX","https://www.smxtech.com/")

Selecting relevant links for https://www.smxtech.com/ by calling gpt-5-nano
Found 12 relevant links


# SMX Company Brochure

---

## About SMX

**Harness Technology to Realize Your Digital Future**

For 30 years, SMX has been a leading technology company dedicated to transforming how businesses and government agencies achieve mission success. Combining deep technical expertise and domain knowledge, SMX empowers public and private sector enterprises to become more effective, innovative, and resilient in the face of evolving challenges.

SMX specializes in digital transformation solutions that span cybersecurity, data analytics, enterprise applications, mission operations & intelligence, modeling, simulation & training, and advanced accelerators like SMX Elevate. Their innovative delivery model offers tailored forward-looking insights and practical solutions to drive secure, critical mission acceleration.

Recent strategic growth includes acquiring cBEYONData, significantly enhancing SMX's digital transformation capabilities and broadening their impact on government operations.

---

## Key Solutions & Services

- **Cybersecurity:** Protecting critical information and infrastructures.
- **Data & Analytics:** Delivering actionable insights for complex challenges.
- **Digital Transformation:** Automating and optimizing mission and business outcomes.
- **Enterprise Applications:** Streamlining operations with scalable solutions.
- **Mission Operations & Intelligence:** Accelerating intelligence with commercial ISR platforms.
- **Modeling, Simulation & Training:** Advanced tools to prepare and train for mission readiness.
- **SMX Elevate AI Labs℠:** Secure AI experimentation environments safeguarding proprietary data.

---

## Customers

SMX delivers mission-critical technology solutions supporting a diverse and prestigious client base, including:

- **Department of Defense & U.S. Military:** U.S. Army, Navy, Air Force, Marine Corps.
- **Federal Government Agencies:** Department of Treasury, FBI, Department of Interior.
- **Commercial & Industry Leaders:** Allergan, Mayo Clinic, Pepperidge Farm, Ancestry, Dole, Choice Logistics.
- **Public and Regulatory Entities:** Large convention centers, city authorities, and various federal regulatory bodies.

SMX is trusted to optimize business outcomes and support complex missions with innovative, scalable, and secure technology.

---

## Company Culture & Awards

At SMX, innovation is a daily expectation. The company nurtures a culture where employees are encouraged to push beyond "what if?" through collaboration, respect for diversity, and shared ownership of impactful projects—ranging from commercial digital transformation to “can’t-fail” national security initiatives.

SMX has garnered multiple accolades, including:

- **Energage Top Workplaces USA** (2023, 2024, 2025)
- **#1 Best Firm to Work For**
- **Top Technology Firm**
- **Health and Wellness Trailblazer**
- **Employer Support of the Guard and Reserve (ESGR) Patriot Award**
- **Military Times Best for Vets Employer**
- Culture Excellence recognitions in Compensation & Benefits, Innovation, Employee Well-Being, and Work-Life Flexibility.

---

## Careers at SMX

SMX offers dynamic career opportunities across various locations including Southern Maryland, Hawaii, Washington D.C., Colorado, and more. The company fosters professional growth through challenging technical projects, diverse teams, and a commitment to innovation.

**Career Perks include:**

- Competitive health benefits (medical, dental, vision)
- Wellness programs and financial wellness support like 401(k) with company match
- Work-life integration initiatives such as paid time off, legal assistance, and identity theft protection
- SkillBridge Fellowship Program supporting bridging veterans into technology careers
- Recognition for being an employer of choice in national surveys for multiple consecutive years

Whether you're a cleared professional, a technologist seeking growth, or a mission-driven individual, SMX provides a platform to innovate and impact critical missions.

---

## Quality & Standards

SMX is committed to quality and continuous improvement with certifications including:

- ISO 9001 (since 2005)
- AS9100D (aviation, space, and defense industries certification)
- CMMI Service & Development Level 3 (since 2022/2023)
- ISO 20000 (IT Service Management)
- ISO 27001 (Information Security Management)

These rigorous standards ensure consistent delivery and secure solutions for complex, mission-critical environments.

---

## Partners

A Premier Partner with major cloud and technology providers, SMX maintains strong alliances to deliver best-in-class solutions:

- Amazon Web Services (AWS Premier Consulting Partner since 2012)
- Microsoft Azure Expert Managed Service Provider & Gold Partner
- Google Cloud Partner and Authorized Reseller
- Oracle Partner

Technology alliances include Appian, Dell, Fortinet, IBM, Palo Alto Networks, Salesforce, ServiceNow, Snowflake, Tableau, and many others—ensuring access to the latest innovations and tools.

---

## Contact SMX

**Corporate Headquarters:** Hollywood, MD  
Email: [info@smxcorp.com](mailto:info@smxcorp.com)  
Careers: [careers@smxcorp.com](mailto:careers@smxcorp.com)  
Sales & Solutions: [sales@smxcorp.com](mailto:sales@smxcorp.com)  

Website: [www.smxcorp.com](https://www.smxcorp.com)  
"Shared Vision. Outcome Assured.®"

---

**SMX**: Accelerate your mission with trusted technology expertise and digital transformation innovations designed for the complex demands of government and commercial sectors. Join SMX in realizing the digital future—securely, efficiently, and at scale.